In [1]:
import numpy as np
from scipy.stats import norm, kurtosis,skew
import matplotlib.pyplot as plt
from matplotlib import pyplot
from statistics import median,mean,stdev
from math import sqrt, ceil

from numpy import sqrt, newaxis
from numpy.fft import irfft, rfftfreq
from numpy.random import normal
from numpy import sum as npsum

import seaborn as sns
import matplotlib.mlab as mlab
from scipy import stats, optimize

np.seterr(under='ignore')
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def grng1(n):
    
    res = n/12
    
    data = np.array(pd.DataFrame(np.random.randn(n) * np.sqrt(res) * np.sqrt(1 / n)).cumsum())
    
    return data

In [ ]:
def Estadisticos(data):
    
    nn,mm = np.shape(data)
    
    Promedio = np.zeros([mm])
    Varianza = np.zeros([mm])
    Skewness = np.zeros([mm])
    Kurtosis = np.zeros([mm])
    
    for t in range(mm):   
        Promedio[t] = np.mean(data[:,t])
        Varianza[t] = np.var(data[:,t])
        Skewness[t] = skew(data[:,t])
        Kurtosis[t] = kurtosis(data[:,t])
        
    serie_esta = pd.DataFrame({'Prom':Promedio,'Vari':Varianza,'Asim':Skewness,'Curt':Kurtosis})
    
    return serie_esta

In [ ]:
def Estadisticos2(data):
  
    Promedio = np.mean(data)
    Varianza = np.var(data)
    Skewness = skew(data)
    Kurtosis = kurtosis(data)
        
    serie_esta = pd.DataFrame({'Prom':[Promedio],'Vari':[Varianza],'Asim':[Skewness],'Curt':[Kurtosis]})
    
    return serie_esta

In [2]:
def Normalizar1(data):
    
    nn,mm = np.shape(data)
    
    serie_norm = np.zeros([len(data),mm])
    Promedio = np.zeros([mm])
    Varianza = np.zeros([mm])
    Skewness = np.zeros([mm])
    Kurtosis = np.zeros([mm])
    
    for t in range(mm):
        for i in range(len(data)):
            serie_norm[i,t] = (data[i,t] - np.min(data[:,t]))/(np.max(data[:,t]) - np.min(data[:,t]))
    
        Promedio[t] = np.mean(serie_norm[:,t])
        Varianza[t] = np.var(serie_norm[:,t])
        Skewness[t] = skew(serie_norm[:,t])
        Kurtosis[t] = kurtosis(serie_norm[:,t])
        
    if len(data) == 64:
        color = 'bo'
    elif len(data) == 128:
        color = 'ro'
    elif len(data) == 256:
        color = 'go'
    elif len(data) == 512:
        color = 'ko'
    elif len(data) == 1024:
        color = 'co'
    elif len(data) == 2048:
        color = 'mo'
    elif len(data) == 4096:
        color = 'yo'
    elif len(data) == 8192:
        color = 'b*'
        
    serie_esta = pd.DataFrame({'Group':str(len(data)),'Prom':Promedio,'Vari':Varianza, \
                                'Asim':Skewness,'Curt':Kurtosis,'Cor':color})    
        
    return serie_norm,serie_esta

In [ ]:
def Normalizar2(data):
    
    serie_norm = np.zeros([len(data)])
    
    for i in range(len(data)):
        serie_norm[i] = (data[i] - np.min(data))/(np.max(data) - np.min(data))
    
    Promedio = np.mean(serie_norm)
    Varianza = np.var(serie_norm)
    Skewness = skew(serie_norm)
    Kurtosis = kurtosis(serie_norm)

    serie_esta = pd.DataFrame({'Prom':[Promedio],'Vari':[Varianza],'Asim':[Skewness],'Curt':[Kurtosis]})  
        
    return serie_norm,serie_esta

In [4]:
def pmodel(noValues=1024, p=0.4999, slope=[]):
    noOrders = int(np.ceil(np.log2(noValues)))
    noValuesGenerated = 2**noOrders
    
    y = np.array([1])
    for n in range(noOrders):
        y = next_step_1d(y, p)
    
    if (slope):
        fourierCoeff = fractal_spectrum_1d(noValues, slope/2)
        meanVal = np.mean(y)
        stdy = np.std(y)
        x = np.fft.ifft(y - meanVal)
        phase = np.angle(x)
        x = fourierCoeff*np.exp(1j*phase)
        x = np.fft.fft(x).real
        x *= stdy/np.std(x)
        x += meanVal
    else:
        x = y
    
    return x[0:noValues], y[0:noValues]

     
def next_step_1d(y, p):
    y2 = np.zeros(y.size*2)
    sign = np.random.rand(1, y.size) - 0.5
    sign /= np.abs(sign)
    y2[0:2*y.size:2] = y + sign*(1-2*p)*y
    y2[1:2*y.size+1:2] = y - sign*(1-2*p)*y
    
    return y2


def fractal_spectrum_1d(noValues, slope):
    ori_vector_size = noValues
    ori_half_size = ori_vector_size//2
    a = np.zeros(ori_vector_size)
    
    for t2 in range(ori_half_size):
        index = t2
        t4 = 1 + ori_vector_size - t2
        if (t4 >= ori_vector_size):
            t4 = t2
        coeff = (index + 1)**slope
        a[t2] = coeff
        a[t4] = coeff
        
    a[1] = 0
    
    return a

In [ ]:
def powerlaw_psd_gaussian(exponent, size, fmin=0):
    """Gaussian (1/f)**beta noise.

    Based on the algorithm in:
    Timmer, J. and Koenig, M.:
    On generating power law noise.
    Astron. Astrophys. 300, 707-710 (1995)

    Normalised to unit variance

    Parameters:
    -----------

    exponent : float
        The power-spectrum of the generated noise is proportional to

        S(f) = (1 / f)**beta
        flicker / pink noise:   exponent beta = 1
        brown noise:            exponent beta = 2

        Furthermore, the autocorrelation decays proportional to lag**-gamma
        with gamma = 1 - beta for 0 < beta < 1.
        There may be finite-size issues for beta close to one.

    shape : int or iterable
        The output has the given shape, and the desired power spectrum in
        the last coordinate. That is, the last dimension is taken as time,
        and all other components are independent.

    fmin : float, optional
        Low-frequency cutoff.
        Default: 0 corresponds to original paper. It is not actually
        zero, but 1/samples.

    Returns
    -------
    out : array
        The samples.


    Examples:
    ---------

    # generate 1/f noise == pink noise == flicker noise
    >>> import colorednoise as cn
    >>> y = cn.powerlaw_psd_gaussian(1, 5)
    """
    
    # Make sure size is a list so we can iterate it and assign to it.
    try:
        size = list(size)
    except TypeError:
        size = [size]
    
    # The number of samples in each time series
    samples = size[-1]
    
    # Calculate Frequencies (we asume a sample rate of one)
    # Use fft functions for real output (-> hermitian spectrum)
    f = rfftfreq(samples)
    
    # Build scaling factors for all frequencies
    s_scale = f
    fmin = max(fmin, 1./samples) # Low frequency cutoff
    ix   = npsum(s_scale < fmin)   # Index of the cutoff
    if ix and ix < len(s_scale):
        s_scale[:ix] = s_scale[ix]
    s_scale = s_scale**(-exponent/2.)
    
    # Calculate theoretical output standard deviation from scaling
    w      = s_scale[1:].copy()
    w[-1] *= (1 + (samples % 2)) / 2. # correct f = +-0.5
    sigma = 2 * sqrt(npsum(w**2)) / samples
    
    # Adjust size to generate one Fourier component per frequency
    size[-1] = len(f)

    # Add empty dimension(s) to broadcast s_scale along last
    # dimension of generated random power + phase (below)
    dims_to_add = len(size) - 1
    s_scale     = s_scale[(newaxis,) * dims_to_add + (Ellipsis,)]
    
    # Generate scaled random power + phase
    sr = normal(scale=s_scale, size=size)
    si = normal(scale=s_scale, size=size)
    
    # If the signal length is even, frequencies +/- 0.5 are equal
    # so the coefficient must be real.
    if not (samples % 2): si[...,-1] = 0
    
    # Regardless of signal length, the DC component must be real
    si[...,0] = 0
    
    # Combine power + corrected phase to Fourier components
    s  = sr + 1J * si
    
    # Transform to real time series & scale to unit variance
    y = irfft(s, n=samples, axis=-1) / sigma
    
    return y

In [7]:
class cullen_frey():
    """
    Create a skewness-curtosis graph based on Cullen and Frey (1999)
    """
    def __init__(self,data,method='unbiased',discrete=False,boot=None,graph=True):
        """
        Parameters
        data: N x 1 list of sample data
        method: 'unbiased' for unbiased estimated values of statistics or 'sample' for sample values.
        discrete: If True, the distribution is considered as discrete.
        boot: If not None, boot values of skewness and kurtosis are plotted from bootstrap samples of data. 
        boot must be fixed in this case to an integer above 10.
        graph: If False, the skewness-kurtosis graph is not plotted.
        """
        
        self.df = data
        self.method = method
        self.discrete = discrete
        self.boot = boot
        self.graph = graph
        
        if not isinstance(self.df,list) or len(np.shape(self.df)) > 1: 
            raise TypeError('Samples must a list with N x 1 dimensions')
        
        if len(self.df) < 4:
            raise ValueError('The number of samples needs to be greater than 4')
            
        if self.boot is not None:    
            if not isinstance(self.boot,int):
                raise ValueError('boot must be integer')
            
        self.stats()
        
        
    def stats(self):
        """
        Evaluates the min, max, median, mean, skewness and kurtosis of data
        """
        if self.method=='unbiased':        
            self.skewdata = skew(self.df, bias=False)
            self.kurtdata = kurtosis(self.df, bias=False)+3

        elif self.method=='sample':
            self.skewdata = skew(self.df, bias=True)
            self.kurtdata = kurtosis(self.df, bias=True)+3

        
        res = [min(self.df),max(self.df),median(self.df),mean(self.df),stdev(self.df),self.skewdata,self.kurtdata]
    
        #Resumo estatístico
        #print(f'min:  {res[0]:.6f}   max: {res[1]:.6f}')
        #print(f'median:  {res[2]:.6f}')
        #print(f'mean:  {res[3]:.6f}')
        #print(f'estimated mean:  {res[4]:.6f}')
        #print(f'estimated skewness:  {res[5]:.6f}')
        #print(f'estimated kurtosis:  {res[6]:.6f}')

        if self.graph:
            self.cullen_frey_graph()
            
    def cullen_frey_graph(self): 
        """
        If graph = True, draws the skewness-kurtosis plot
        """
         
        if self.boot is not None:
            if self.boot < 10:
                raise ValueError('boot must be greater than 10')

            n = len(self.df)

            nrow = n
            ncol = self.boot
            databoot = np.reshape(np.random.choice(self.df, size=n*self.boot, replace=True),(nrow,ncol)) 

            s2boot = (skew(pd.DataFrame(databoot)))**2
            kurtboot = kurtosis(pd.DataFrame(databoot))+3

            kurtmax = max(10,ceil(max(kurtboot)))
            xmax = max(4,ceil(max(s2boot)))

        else:
            kurtmax = max(10,ceil(self.kurtdata))
            xmax = max(4,ceil(self.skewdata**2))

        ymax = kurtmax-1
        
        # If discrete = False
        if not self.discrete:
            #Beta distribution
            p = np.exp(-100)
            lq = np.arange(-100,100.1,0.1)
            q = np.exp(lq)
            s2a = (4*(q-p)**2*(p+q+1))/((p+q+2)**2*p*q)
            ya = kurtmax-(3*(p+q+1)*(p*q*(p+q-6)+2*(p+q)**2)/(p*q*(p+q+2)*(p+q+3)))
            p = np.exp(100)
            lq = np.arange(-100,100.1,0.1)
            q = np.exp(lq)
            s2b = (4*(q-p)**2*(p+q+1))/((p+q+2)**2*p*q)
            yb = kurtmax-(3*(p+q+1)*(p*q*(p+q-6)+2*(p+q)**2)/(p*q*(p+q+2)*(p+q+3)))
            s2 = [*s2a,*s2b]
            y = [*ya,*yb]

            #Gama distribution
            lshape_gama = np.arange(-100,100,0.1)
            shape_gama = np.exp(lshape_gama)
            s2_gama = 4/shape_gama
            y_gama = kurtmax-(3+6/shape_gama) 

            #Lognormal distribution
            lshape_lnorm = np.arange(-100,100,0.1)
            shape_lnorm = np.exp(lshape_lnorm)
            es2_lnorm = np.exp(shape_lnorm**2, dtype=np.float64)
            s2_lnorm = (es2_lnorm+2)**2*(es2_lnorm-1)
            y_lnorm = kurtmax-(es2_lnorm**4+2*es2_lnorm**3+3*es2_lnorm**2-3)

            plt.figure(figsize=(12,9))
            
            #observations
            obs = plt.scatter(self.skewdata**2,kurtmax-self.kurtdata,s=200, c='blue', 
                              label='Observation',zorder=10)
            #beta
            beta = plt.fill(s2,y,color='lightgrey',alpha=0.6, label='beta', zorder=0)
            #gama
            gama = plt.plot(s2_gama,y_gama, '--', c='k', label='gama')
            #lognormal
            lnormal = plt.plot(s2_lnorm,y_lnorm, c='k', label='lognormal')
            
            if self.boot is not None:
                #bootstrap 
                bootstrap = plt.scatter(s2boot,kurtmax-kurtboot,marker='$\circ$',c='orange',s=50, 
                                        label='Bootstrap values', zorder=5)
                legenda1 = plt.legend(handles=[bootstrap],loc=(xmax*0.1065,ymax*0.101), 
                                      labelspacing=2, frameon=False)
                plt.gca().add_artist(legenda1)
            

            #markers
            normal = plt.scatter(0,kurtmax-3, marker=(8,2,0),s=200,c='k',label='normal',zorder=5)

            uniform = plt.scatter(0,kurtmax-9/5, marker='$\\bigtriangleup$',s=200,c='k',\
                                  label='uniform',zorder=5)   

            exp_dist = plt.scatter(2**2,kurtmax-9, marker='$\\bigotimes$',s=200,c='k',\
                                   label='exponential',zorder=5) 

            logistic = plt.scatter(0,kurtmax-4.2, marker='+',s=200,c='k',label='logistic',zorder=5)


            #Adjusting the axis
            yax = [str(kurtmax - i) for i in range(0,ymax+1)]
            plt.xlim(-1, xmax+0.4)
            plt.ylim(-1, ymax+0.08)
            plt.yticks(np.arange(0,ymax+2,5),labels=yax,fontsize=14)
            plt.xticks(fontsize=14)

            #Adding the labels
            plt.xlabel('square of skewness', fontsize=18)
            plt.ylabel('kurtosis', fontsize=18)
            plt.title('Cullen and Frey graph', fontsize=20) 

            #Adding the legends
            legenda2 = plt.legend(handles=[obs],loc='upper center', labelspacing=1, frameon=False)
            plt.gca().add_artist(legenda2)

            plt.legend(handles=[normal,uniform,exp_dist,logistic,beta[0],lnormal[0],gama[0]], 
                       title='Theoretical distributions',loc='lower left',labelspacing=1.4,frameon=False,
                       prop={'size': 16})

            plt.show()
        
        #If discrete = True
        else:
            # negbin distribution
            p = np.exp(-10)
            lr = np.arange(-100,100,0.1)
            r = np.exp(lr)
            s2a = (2-p)**2/(r*(1-p))
            ya = kurtmax-(3+6/r+p**2/(r*(1-p)))
            p = 1-np.exp(-10)
            lr = np.arange(100,-100,-0.1)
            r = np.exp(lr)
            s2b = (2-p)**2/(r*(1-p))
            yb = kurtmax-(3+6/r+p**2/(r*(1-p)))
            s2_negbin = [*s2a,*s2b]
            y_negbin = [*ya,*yb]
            
            # poisson distribution
            llambda = np.arange(-100,100,0.1)
            lambda_ = np.exp(llambda)
            s2_poisson = 1/lambda_
            y_poisson = kurtmax-(3+1/lambda_)
            
            plt.figure(figsize=(12,9))          
            
            #observations
            obs = plt.scatter(self.skewdata**2,kurtmax-self.kurtdata,s=200, c='blue', 
                              label='Observation',zorder=10)
            
            #negative binomial
            negbin = plt.fill(s2_negbin,y_negbin,color='lightgrey',alpha=0.6, \
                              label='negative binomial', zorder=0)
            
            #poisson
            poisson = plt.plot(s2_poisson,y_poisson, '--', c='k', label='poisson')
            
            if self.boot is not None:
                #bootstrap 
                bootstrap = plt.scatter(s2boot,kurtmax-kurtboot,marker='$\circ$',c='orange',s=50, 
                                        label='Bootstrap values', zorder=5)
                legenda2 = plt.legend(handles=[bootstrap],loc=(xmax*0.1065,ymax*0.101), 
                                      labelspacing=2, frameon=False)
                plt.gca().add_artist(legenda2)
            
            #markers
            normal = plt.scatter(0,kurtmax-3, marker=(8,2,0),s=400,c='k',label='normal',zorder=5)
            
            #adjusting the axis
            yax = [str(kurtmax - i) for i in range(0,ymax+1)]
            plt.xlim(-1, xmax+0.4)
            plt.ylim(-1, ymax+1)
            
            plt.yticks(np.arange(0,ymax+2,4),labels=yax)
            #plt.yticks(list(range(0,ymax+1,2)),labels=yax)

            #adding the labels
            plt.xlabel('square of skewness', fontsize=18)
            plt.ylabel('kurtosis', fontsize=18)
            plt.title('Cullen and Frey graph', fontsize=20) 
            
            #adding the legends
            legenda1 = plt.legend(handles=[obs],loc='upper center', labelspacing=1, frameon=False)
            plt.gca().add_artist(legenda1)

            plt.legend(handles=[normal,negbin[0],poisson[0]],title='Theoretical distributions',\
                       loc='upper right',labelspacing=1.4,frameon=False)
            
            plt.show()

In [1]:
#Gerador de Mapa Logístico Caótico 1D: Atrator e Série Temporal
#1D Chaotic Logistic Map Generator: Attractor and Time Series
#Reinaldo R. Rosa - LABAC-INPE
#Version 1.0 for CAP239-2020

#chaotic logistic map is f(x) = rho*x*(1-x)  with rho in (3.81,4.00)
def Logistic(rho,tau,x,y):

    return rho*x*(1.0-x), tau*x

In [ ]:
#Gerador de Mapa Logístico Caótico 2D (Henon Map): Atrator e Série Temporal
#2D Chaotic Logistic Map Generator (Henon Map): Attractor and Time Series
#Reinaldo R. Rosa - LABAC-INPE
#Version 1.0 for CAP239-2020

#2D Henon logistic map is noise-like with "a" in (1.350,1.420) and "b" in (0.210,0.310)
def HenonMap(a,b,x,y):

    return y + 1.0 - a *x*x, b * x

In [1]:
def psd(data):

    """Calcula o PSD de uma série temporal."""

    # Define um intervalo para realizar o ajuste da reta

    INICIO = 10

    FIM = 200

    # O vetor com o tempo é o tamanho do número de pontos

    N = len(data)

    tempo = np.arange(len(data))

    # Define a frequência de amostragem

    dt = (tempo[-1] - tempo[0] / (N - 1))

    fs = 1 / dt

    # Calcula o PSD utilizando o MLAB

    power, freqs = mlab.psd(data, Fs = fs, NFFT = N, scale_by_freq = False)

    # Calcula a porcentagem de pontos utilizados na reta de ajuste

    totalFrequencias = len(freqs)

    totalPSD = FIM - INICIO

    porcentagemPSD = int(100 * totalPSD / totalFrequencias)

    # Seleciona os dados dentro do intervalo de seleção

    xdata = freqs[INICIO:FIM]

    ydata = power[INICIO:FIM]

    # Simula o erro

    yerr = 0.2 * ydata

    # Define uma função para calcular a Lei de Potência

    powerlaw = lambda x, amp, index: amp * (x**index)

    # Converte os dados para o formato LOG

    logx = np.log10(xdata)

    logy = np.log10(ydata)

    # Define a função para realizar o ajuste

    fitfunc = lambda p, x: p[0] + p[1] * x

    errfunc = lambda p, x, y, err: (y - fitfunc(p, x)) / err    

    logyerr = yerr / ydata

    # Calcula a reta de ajuste

    pinit = [1.0, -1.0]

    out = optimize.leastsq(errfunc, pinit, args = (logx, logy, logyerr), full_output = 1)    

    pfinal = out[0]

    covar = out[1]

    index = -1*pfinal[1]

    amp = 10.0 ** pfinal[0]

    indexErr = np.sqrt(covar[0][0])

    ampErr = np.sqrt(covar[1][1]) * amp

    # Retorna os valores obtidos

    return freqs, power, xdata, ydata, amp, index, powerlaw, INICIO, FIM

In [ ]:
def psd2(data):

    """Calcula o PSD de uma série temporal."""

    # Define um intervalo para realizar o ajuste da reta

    INICIO = 10

    FIM = 200

    # O vetor com o tempo é o tamanho do número de pontos

    N = len(data)

    tempo = np.arange(len(data))

    # Define a frequência de amostragem

    dt = (tempo[-1] - tempo[0] / (N - 1))

    fs = 1 / dt

    # Calcula o PSD utilizando o MLAB

    power, freqs = mlab.psd(data, Fs = fs, NFFT = N, scale_by_freq = False)

    # Calcula a porcentagem de pontos utilizados na reta de ajuste

    totalFrequencias = len(freqs)

    totalPSD = FIM - INICIO

    porcentagemPSD = int(100 * totalPSD / totalFrequencias)

    # Seleciona os dados dentro do intervalo de seleção

    xdata = freqs[INICIO:FIM]

    ydata = power[INICIO:FIM]

    # Simula o erro

    yerr = 0.2 * ydata

    # Define uma função para calcular a Lei de Potência

    powerlaw = lambda x, amp, index: amp * (x**index)

    # Converte os dados para o formato LOG

    logx = np.log10(xdata)

    logy = np.log10(ydata)

    # Define a função para realizar o ajuste

    fitfunc = lambda p, x: p[0] + p[1] * x

    errfunc = lambda p, x, y, err: (y - fitfunc(p, x)) / err    

    logyerr = yerr / ydata

    # Calcula a reta de ajuste

    pinit = [1.0, -1.0]

    out = optimize.leastsq(errfunc, pinit, args = (logx, logy, logyerr), full_output = 1)    

    pfinal = out[0]

    covar = out[1]

    index = -1*pfinal[1]

    amp = 10.0 ** pfinal[0]

    indexErr = np.sqrt(covar[0][0])

    ampErr = np.sqrt(covar[1][1]) * amp

    # Retorna os valores obtidos

    return freqs, power, xdata, ydata, amp, index, powerlaw, INICIO, FIM

In [2]:
def dfa1d(timeSeries, grau):

    """Calcula o DFA 1D (adaptado de Physionet), onde a escala cresce

    de acordo com a variável 'Boxratio'. Retorna o array 'vetoutput', 

    onde a primeira coluna é o log da escala S e a segunda coluna é o

    log da função de flutuação."""


    # 1. A série temporal {Xk} com k = 1, ..., N é integrada na chamada função perfil Y(k)

    x = np.mean(timeSeries)

    timeSeries = timeSeries - x

    yk = np.cumsum(timeSeries)

    tam = len(timeSeries)


    # 2. A série (ou perfil) Y(k) é dividida em N intervalos não sobrepostos de tamanho S

    sf = np.ceil(tam / 4).astype(np.int)

    boxratio = np.power(2.0, 1.0 / 8.0)

    vetoutput = np.zeros(shape = (1,2))


    s = 4

    while s <= sf:        

        serie = yk        

        if np.mod(tam, s) != 0:

            l = s * int(np.trunc(tam/s))

            serie = yk[0:l]			

        t = np.arange(s, len(serie), s)

        v = np.array(np.array_split(serie, t))

        l = len(v)

        x = np.arange(1, s + 1)


        # 3. Calcula-se a variância para cada segmento v = 1,…, n_s:

        p = np.polynomial.polynomial.polyfit(x, v.T, grau)

        yfit = np.polynomial.polynomial.polyval(x, p)

        vetvar = np.var(v - yfit)


    # 4. Calcula-se a função de flutuação DFA como a média das variâncias de cada intervalo

        fs = np.sqrt(np.mean(vetvar))

        vetoutput = np.vstack((vetoutput,[s, fs]))


        # A escala S cresce numa série geométrica

        s = np.ceil(s * boxratio).astype(np.int)


    # Array com o log da escala S e o log da função de flutuação   

    vetoutput = np.log10(vetoutput[1::1,:])


    # Separa as colunas do vetor 'vetoutput'

    x = vetoutput[:,0]

    y = vetoutput[:,1]


    # Regressão linear

    slope, intercept, _, _, _ = stats.linregress(x, y)


    # Calcula a reta de inclinação

    predict_y = intercept + slope * x


    # Calcula o erro

    pred_error = y - predict_y


    # Retorna o valor do ALFA, o vetor 'vetoutput', os vetores X e Y,

    # o vetor com os valores da reta de inclinação e o vetor de erros

    return slope, vetoutput, x, y, predict_y, pred_error

In [ ]:
def getHurstByUpscaling(dx, normType_p = np.inf, isDFA = 1, normType_q = 1.0):
    ## Some initialiation
    dx_len = len(dx)
    
    # We have to reserve the most major scale for shifts, so we divide the data
    # length by two. (As a result, the time measure starts from 2.0, not from
    # 1.0, see below.)
    dx_len = np.int(dx_len / 2)
    
    dx_shift = np.int(dx_len / 2)
    
    nScales = np.int(np.round(np.log2(dx_len)))    # Number of scales involved. P.S. We use 'round()' to prevent possible malcomputing of the logarithms
    j = 2 ** (np.arange(1, nScales + 1) - 1) - 1
    
    meanDataMeasure = np.zeros(nScales)
    
    ## Computing the data measure
    for ji in range(1, nScales + 1):
        # At the scale 'j(ji)' we deal with '2 * (j(ji) + 1)' elements of the data 'dx'
        dx_k_len = 2 * (j[ji - 1] + 1)
        n = np.int(dx_len / dx_k_len)
        
        dx_leftShift = np.int(dx_k_len / 2)
        dx_rightShift = np.int(dx_k_len / 2)
        
        for k in range(1, n + 1):
            # We get a portion of the data of the length '2 * (j(ji) + 1)' plus the data from the left and right boundaries
            dx_k_withShifts = dx[(k - 1) * dx_k_len + 1 + dx_shift - dx_leftShift - 1 : k * dx_k_len + dx_shift + dx_rightShift]
            
            # Then we perform free upscaling and, using the above-selected data (provided at the scale j = 0),
            # compute the velocities at the scale 'j(ji)'
            j_dx = np.convolve(dx_k_withShifts, np.ones(dx_rightShift), 'valid')
            
            # Then we compute the accelerations at the scale 'j(ji) + 1'
            r = (j_dx[1 + dx_rightShift - 1 : ] - j_dx[1 - 1 : -dx_rightShift]) / 2.0
            
            # Finally, we compute the range ...
            if (normType_p == 0):
                R = np.max(r[2 - 1 : ]) - np.min(r[2 - 1 : ])
            elif (np.isinf(normType_p)):
                R = np.max(np.abs(r[2 - 1 : ]))
            else:
                R = (np.sum(r[2 - 1 : ] ** normType_p) / len(r[2 - 1 : ])) ** (1.0 / normType_p)
            # ... and the normalisation factor ("standard deviation")
            S = np.sqrt(np.sum(np.abs(np.diff(r)) ** 2.0) / (len(r) - 1))
            if (isDFA == 1):
                S = 1.0
            
            meanDataMeasure[ji - 1] += (R / S) ** normType_q
        meanDataMeasure[ji - 1] = (meanDataMeasure[ji - 1] / n) ** (1.0 / normType_q)
    
    # We pass from the scales ('j') to the time measure; the time measure at the scale j(nScales) (the most major one)
    # is assumed to be 2.0, while it is growing when the scale is tending to j(1) (the most minor one).
    # (The scale j(nScales)'s time measure is NOT equal to 1.0, because we reserved the highest scale for shifts
    # in the very beginning of the function.)
    timeMeasure = 2.0 * dx_len / (2 * (j + 1))
    
    scales = j + 1
    
    return [timeMeasure, meanDataMeasure, scales]

#MFDFA-Analytics-by-SKDataScience
#multifractal DFA singularity spectra - module 02
#Version 3.0 - Modified by R.R.Rosa - Dec 2018 - mfdfa_ss_m2.py
# This code implements a modification of the first-order multifractal analysis algorithm. It is based on the
# corresponding unifractal analysis technique described in [1]. It computes the Lipschitz-Holder multifractal
# singularity spectrum, as well as the minimum and maximum generalised Hurst exponents [2, 3].
#
# At the input, 'dx' is a time series of increments of the physical observable 'x(t)', of the length equal to an
# integer power of two greater than two (i.e. 4, 8, 16, 32, etc.), 'normType' is any real greater than or
# equal to one specifying the p-norm, 'isDFA' is a boolean value prescribing to use either the DFA-based algorithm or
# the standard Hurst (a.k.a. R/S) analysis, 'isNormalised' is a boolean value prescribing either to normalise the
# intermediate range-to-deviation (R/S) expression or to proceed computing without normalisation.
#
# At the output, 'timeMeasure' is the time measure of the data's support at different scales, 'dataMeasure' is
# the data measure at different scales computed for each value of the variable q-norm, 'scales' is the scales at which
# the data measure is computed, 'stats' is the structure containing MF-DFA statistics, while 'q' is the values of the
# q-norm used.
#
# Similarly to unifractal analysis (see getHurstByUpscaling()), the time measure is computed merely for an alternative
# representation of the dependence 'dataMeasure(q, scales) ~ scales ^ -tau(q)'.

def getMSSByUpscaling(dx, normType = np.inf, isDFA = 1, isNormalised = 1):
    ## Some initialiation
    aux_eps = np.finfo(float).eps
    
    # We prepare an array of values of the variable q-norm
    aux = [-16.0, -8.0, -4.0, -2.0, -1.0, -0.5, -0.0001, 0.0, 0.0001, 0.5, 0.9999, 1.0, 1.0001, 2.0, 4.0, 8.0, 16.0, 32.0]
    nq = len(aux)
    
    q = np.zeros((nq, 1))
    q[:, 1 - 1] = aux
    
    dx_len = len(dx)
    
    # We have to reserve the most major scale for shifts, so we divide the data
    # length by two. (As a result, the time measure starts from 2.0, not from
    # 1.0, see below.)
    dx_len = np.int(dx_len / 2)
    
    dx_shift = np.int(dx_len / 2)
    
    nScales = np.int(np.round(np.log2(dx_len)))    # Number of scales involved. P.S. We use 'round()' to prevent possible malcomputing of the logarithms
    j = 2 ** (np.arange(1, nScales + 1) - 1) - 1
    
    dataMeasure = np.zeros((nq, nScales))
    
    ## Computing the data measures in different q-norms
    for ji in range(1, nScales + 1):
        # At the scale 'j(ji)' we deal with '2 * (j(ji) + 1)' elements of the data 'dx'
        dx_k_len = 2 * (j[ji - 1] + 1)
        n = np.int(dx_len / dx_k_len)
        
        dx_leftShift = np.int(dx_k_len / 2)
        dx_rightShift = np.int(dx_k_len / 2)
        
        R = np.zeros(n)
        S = np.ones(n)
        for k in range(1, n + 1):
            # We get a portion of the data of the length '2 * (j(ji) + 1)' plus the data from the left and right boundaries
            dx_k_withShifts = dx[(k - 1) * dx_k_len + 1 + dx_shift - dx_leftShift - 1 : k * dx_k_len + dx_shift + dx_rightShift]
            
            # Then we perform free upscaling and, using the above-selected data (provided at the scale j = 0),
            # compute the velocities at the scale 'j(ji)'
            j_dx = np.convolve(dx_k_withShifts, np.ones(dx_rightShift), 'valid')
            
            # Then we compute the accelerations at the scale 'j(ji) + 1'
            r = (j_dx[1 + dx_rightShift - 1 : ] - j_dx[1 - 1 : -dx_rightShift]) / 2.0
            
            # Finally we compute the range ...
            if (normType == 0):
                R[k - 1] = np.max(r[2 - 1 : ]) - np.min(r[2 - 1 : ])
            elif (np.isinf(normType)):
                R[k - 1] = np.max(np.abs(r[2 - 1 : ]))
            else:
                R[k - 1] = (np.sum(r[2 - 1 : ] ** normType) / len(r[2 - 1 : ])) ** (1.0 / normType)
            # ... and the normalisation factor ("standard deviation")
            if (isDFA == 0):
                S[k - 1] = np.sqrt(np.sum(np.abs(np.diff(r)) ** 2.0) / (len(r) - 1))
    
        if (isNormalised == 1):      # Then we either normalise the R / S values, treating them as probabilities ...
            p = np.divide(R, S) / np.sum(np.divide(R, S))
        else:                        # ... or leave them unnormalised ...
            p = np.divide(R, S)
          # ... and compute the measures in the q-norms
        for k in range(1, n + 1):
            # This 'if' is needed to prevent measure blow-ups with negative values of 'q' when the probability is close to zero
            if (p[k - 1] < 1000.0 * aux_eps):
                continue
            
            dataMeasure[:, ji - 1] = dataMeasure[:, ji - 1] + np.power(p[k - 1], q[:, 1 - 1])

    # We pass from the scales ('j') to the time measure; the time measure at the scale j(nScales) (the most major one)
    # is assumed to be 2.0, while it is growing when the scale is tending to j(1) (the most minor one).
    # (The scale j(nScales)'s time measure is NOT equal to 1.0, because we reserved the highest scale for shifts
    # in the very beginning of the function.)
    timeMeasure = 2.0 * dx_len / (2 * (j + 1))
    
    scales = j + 1
    
    ## Determining the exponents 'tau' from 'dataMeasure(q, timeMeasure) ~ timeMeasure ^ tau(q)'
    tau = np.zeros((nq, 1))
    log10tm = np.log10(timeMeasure)
    log10dm = np.log10(dataMeasure)
    log10tm_mean = np.mean(log10tm)
    
    # For each value of the q-norm we compute the mean 'tau' over all the scales
    for qi in range(1, nq + 1):
        tau[qi - 1, 1 - 1] = np.sum(np.multiply(log10tm, (log10dm[qi - 1, :] - np.mean(log10dm[qi - 1, :])))) / np.sum(np.multiply(log10tm, (log10tm - log10tm_mean)))

    ## Finally, we only have to pass from 'tau(q)' to its conjugate function 'f(alpha)'
    # In doing so, first we find the Lipschitz-Holder exponents 'alpha' (represented by the variable 'LH') ...
    aux_top = (tau[2 - 1] - tau[1 - 1]) / (q[2 - 1] - q[1 - 1])
    aux_middle = np.divide(tau[3 - 1 : , 1 - 1] - tau[1 - 1 : -1 - 1, 1 - 1], q[3 - 1 : , 1 - 1] - q[1 - 1 : -1 - 1, 1 - 1])
    aux_bottom = (tau[-1] - tau[-1 - 1]) / (q[-1] - q[-1 - 1])
    LH = np.zeros((nq, 1))
    LH[:, 1 - 1] = -np.concatenate((aux_top, aux_middle, aux_bottom))
    # ... and then compute the conjugate function 'f(alpha)' itself
    f = np.multiply(LH, q) + tau

    ## The last preparations
    # We determine the minimum and maximum values of 'alpha' ...
    LH_min = LH[-1, 1 - 1]
    LH_max = LH[1 - 1, 1 - 1]
    # ... and find the minimum and maximum values of another multifractal characteristic, the so-called
    # generalised Hurst (or DFA) exponent 'h'. (These parameters are computed according to [2, p. 27].)
    h_min = -(1.0 + tau[-1, 1 - 1]) / q[-1, 1 - 1]
    h_max = -(1.0 + tau[1 - 1, 1 - 1]) / q[1 - 1, 1 - 1]
    
    stats = {'tau':       tau,
        'LH':        LH,
            'f':         f,
                'LH_min':    LH_min,
                    'LH_max':    LH_max,
                        'h_min':     h_min,
                            'h_max':     h_max}
    
    return [timeMeasure, dataMeasure, scales, stats, q]

In [ ]:
########################################################################################################################
# Compute and plot Detrended Fluctuation Analysis
#
# Adapted from:
# https://github.com/reinaldo-rosa-inpe/cap239/blob/f92f84710bb8438e461d2bf5d237e93bb7238d3e/Codigos/Specplus.py
# Written by Paulo Giovani, Reinaldo Roberto Rosa, Murilo da Silva Dantas
#
# Adapted by Rian Koja to publish in a GitHub repository with GPL licence for this specific file.
########################################################################################################################

# ------------------------------------------------------------------------
# Specplus.pyplot
# ------------------------------------------------------------------------
# ---------------------------------------------------------------------
# Computes PSD of a time series
# Optionally receives an interval for the linear regression step
# ---------------------------------------------------------------------
def psd(data, init=0, final=None):
    n = len(data)
    time = np.arange(n)

    # If "final" is not given, use length of data
    if final is None:
        final = n

    # Define sampling frequency:
    dt = (time[-1] - time[0] / (n - 1))
    fs = 1 / dt

    # Compute PSD with MLAB:
    power, freqs = mlab.psd(data, Fs=fs, NFFT=n, scale_by_freq=False)

    # Select data within selction interval
    xdata = freqs[init:final]
    ydata = power[init:final]

    # Simulate error
    yerr = 0.2 * ydata

    # Find logarithms of data:
    logx = np.log10(xdata)
    logy = np.log10(ydata)

    logyerr = yerr / ydata

    # Compute line fit:
    pinit = [1.0, -1.0]
    out = optimize.leastsq(errfunc, pinit, args=(logx, logy, logyerr), full_output=True)
    pfinal = out[0]
    index = pfinal[1]
    amp = 10.0 ** pfinal[0]

    # Returns obtained values
    return freqs, power, xdata, ydata, amp, index, init, final


# Define functions to perform data fit:
def fitfunc(p, x):
    return p[0] + p[1] * x


def errfunc(p, x, y, err):
    return (y - fitfunc(p, x)) / err


# Define a function to compute a Power Law:
def powerlaw(x, amp, index):
    return amp * (x ** index)


# ---------------------------------------------------------------------
# Compute 1D DFA for the time series
# ---------------------------------------------------------------------
def dfa1d(time_series, grau):
    # Compute 1D DFA (adapted from Physionet), where the sclae frows according to 'Boxratio'. Returns the array
    # 'vetoutput', where the first column is the logarithm of S scale and the second column is the logarithm of the
    # fluctuation function

    # 1. The time series {Xk} with k = 1, ..., N is integrated into the profile function Y(k)
    x = np.mean(time_series)
    time_series = time_series - x
    yk = np.cumsum(time_series)
    tam = len(time_series)

    # 2. The (or profile) Y(k) is divided into N non-overlapping intervals of size S
    sf = np.ceil(tam / 4).astype(np.int)
    boxratio = np.power(2.0, 1.0 / 8.0)
    vetoutput = np.zeros(shape=(1, 2))
    s = 4

    while s <= sf:
        serie = yk
        if np.mod(tam, s) != 0:
            aux = s * int(np.trunc(tam / s))
            serie = yk[0:aux]
        t = np.arange(s, len(serie), s)
        v = np.array(np.array_split(serie, t))
        x = np.arange(1, s + 1)

        # 3. Compute variance for each segment v = 1,…, n_s:
        p = np.polynomial.polynomial.polyfit(x, v.T, grau)
        yfit = np.polynomial.polynomial.polyval(x, p)
        vetvar = np.var(v - yfit)

        # 4. Compute the the fluctuation function of the DFA as the average of the variances in each interval
        fs = np.sqrt(np.mean(vetvar))
        vetoutput = np.vstack((vetoutput, [s, fs]))

        # S scale grows in geometric series
        s = np.ceil(s * boxratio).astype(np.int)

    # Array with S scale log values and fluctuation function log values
    vetoutput = np.log10(vetoutput[1::1, :])

    # Split the columns of 'vetoutput'
    x = vetoutput[:, 0]
    y = vetoutput[:, 1]

    # Linear Regression
    slope, intercept, _, _, _ = stats.linregress(x, y)

    # Compute line
    predict_y = intercept + slope * x

    # Compute error
    pred_error = y - predict_y
    # Returns the alpha value (slope), the 'vetoutput' vector, the X and Y vectors,
    # a vector with line values, and the error vector
    return slope, vetoutput, x, y, predict_y, pred_error


# ---------------------------------------------------------------------
# Main section
# ---------------------------------------------------------------------
def main(data):
    # Disble numpy errors and warnings
    np.seterr(divide='ignore', invalid='ignore', over='ignore')

    # -----------------------------------------------------------------
    # General plot parameters:
    # -----------------------------------------------------------------

    # Define subplots
    fig = plt.figure(figsize=(22,20),dpi=200)
    fig.subplots_adjust(hspace=.3, wspace=.2)

    # Font sizes:
    size_font_axis_x = 10
    size_font_axis_y = 10
    size_font_title = 8
    size_font_main = 15

    # Main title
    title_main = 'Time Series Spectral Analysis'

    # -----------------------------------------------------------------
    # Plot original series
    # -----------------------------------------------------------------

    # Define plot colors
    cor_serie_original = 'r'
    # Titles for axes on the original series plot
    text_axis_x = 'Time'
    text_axis_y = 'Amplitude'
    text_title_original = 'Original Time Series Data'

    # Plot original data series:
    fig_handle = fig.add_subplot(2, 1, 1)
    fig_handle.plot(data, '-', color=cor_serie_original)
    fig_handle.set_title(text_title_original, fontsize=size_font_title)
    fig_handle.set_xlabel(text_axis_x, fontsize=size_font_axis_x)
    fig_handle.set_ylabel(text_axis_y, fontsize=size_font_axis_y)
    fig_handle.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
    fig_handle.grid()

    # -----------------------------------------------------------------
    # Compute and plot PSD
    # -----------------------------------------------------------------

    # Compute PSD
    freqs, power, xdata, ydata, amp, index, init, fim = psd(data)

    # Beta value is equivalent to the index:
    beta = index

    # Define plot colors:
    cor_psd1 = 'k'
    cor_psd2 = 'navy'

    # PSD axes titles:
    texto_psdx = 'Frequency (Hz)'
    texto_psdy = 'Power'
    texto_titulo_psd = r'Power Spectrum Density $\beta$ = '

    # -----------------------------------------------------------------
    # Compute and plot DFA
    # -----------------------------------------------------------------

    # Compute 1D DFA
    alfa, vetoutput, x, y, reta, error = dfa1d(data, 1)
    # From Neelakshi et. al. (2019) "Spectral fluctuation analysis of ionospheric inhomogeneities over Brazilian
    # territory Part II: EF valley region plasma instabilities"
    # "S. Heneghan and McDarby (2000) established an equivalence relation between the PSD exponent, b, and the DFA
    # exponent, a, given by beta =  2 * alpha - 1. Kiyono (2015) showed that this relationship is valid for
    # the higher order DFA subject to the constraint  0<a<m+1, where m is the order of detrending polynomial in the DFA"
    beta_theoretical = 2 * alfa - 1

    # Plot PSD

    fig_handle = fig.add_subplot(2, 2, 3)

    fig_handle.plot(freqs, power, '-', color=cor_psd1, alpha=0.7)
    fig_handle.plot(xdata, ydata, color=cor_psd2, alpha=0.8)
    fig_handle.axvline(freqs[init], color=cor_psd2, linestyle='--')
    fig_handle.axvline(freqs[-1], color=cor_psd2, linestyle='--')
    fig_handle.plot(xdata, powerlaw(xdata, amp, index), 'r-', linewidth=1.5, label='$%.2f$' % beta)
    fig_handle.set_xlabel(texto_psdx, fontsize=size_font_axis_x)
    fig_handle.set_ylabel(texto_psdy, fontsize=size_font_axis_y)
    fig_handle.set_title(texto_titulo_psd + r'%.2f (Theoretical $\beta$ = 2$\alpha$ +1 = %.2f)' % (beta,
                                                                                                   beta_theoretical),
                         loc='center', fontsize=size_font_title)
    fig_handle.set_yscale('log')
    fig_handle.set_xscale('log')
    fig_handle.grid()

    # Checks if DFA has valid value. If so, proceed with plot:

    if not math.isnan(alfa):

        # Define plot colors:
        cor_dfa = 'darkmagenta'

        # DFA axes title:
        texto_dfax = '$log_{10}$ (s)'
        texto_dfay = '$log_{10}$ F(s)'
        texto_titulo_dfa = r'Detrended Fluctuation Analysis $\alpha$ = '

        # Plot DFA
        fig_dfa = fig.add_subplot(2, 2, 4)
        fig_dfa.plot(x, y, 's', color=cor_dfa, markersize=4, markeredgecolor='r', markerfacecolor='None', alpha=0.8)
        fig_dfa.plot(x, reta, '-', color=cor_dfa, linewidth=1.5)
        fig_dfa.set_title(texto_titulo_dfa + '%.4f' % alfa, loc='center', fontsize=size_font_title)
        fig_dfa.set_xlabel(texto_dfax, fontsize=size_font_axis_x)
        fig_dfa.set_ylabel(texto_dfay, fontsize=size_font_axis_y)
        fig_dfa.grid()

    else:
        fig_dfa = fig.add_subplot(2, 2, 4)
        fig_dfa.set_title('Detrended Fluctuation Analysis $\alpha$ = ' + 'N.A.', loc='center',
                          fontsize=size_font_title)
        fig_dfa.grid()

    # Draw and save figure (avoid showing to prevent blocking)
    plt.suptitle(title_main, fontsize=size_font_main)
    img_filename = 'ANALYSIS_PSD_DFA_2.png'
    fig.set_size_inches(10, 5)
    plt.savefig(img_filename, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.draw()

    return alfa, beta_theoretical

In [1]:
def Area_noise():
    noise = np.array([[1.44769616, 1.82014602, 0.14640649, 0.66346162],
                       [1.40956072, 1.1496771 , 2.73870097, 2.14094958],
                       [1.19081834, 1.1496771 , 1.62616893, 2.14094958],
                       [1.55327776, 1.40956072, 2.38220716, 2.73870097],
                       [1.40997864, 1.44769616, 0.16779165, 0.14640649],
                       [1.40997864, 1.19081834, 0.16779165, 1.62616893],
                       [1.63758865, 1.82014602, 1.94768922, 0.66346162],
                       [1.63758865, 1.55327776, 1.94768922, 2.38220716]])

    Color_noise_pink = np.array([[1.02389761, 1.02804343, 1.23856761, 0.97903838],
                                [0.96329599, 0.94043926, 1.23854284, 0.87054758],
                                [0.96329599, 1.02389761, 1.23854284, 1.23856761],
                                [0.95940765, 0.94043926, 0.82558347, 0.87054758],
                                [0.95940765, 0.99140129, 0.82558347, 0.80742159],
                                [1.01411267, 1.02804343, 0.86101667, 0.97903838],
                                [1.01411267, 0.99140129, 0.86101667, 0.80742159]])

    Color_noise_red = np.array([[1.49828585, 1.41541209, 2.40534815, 1.94596806],
                               [1.49828585, 1.54609464, 2.40534815, 1.94690582],
                               [1.47696749, 1.41541209, 1.70398688, 1.94596806],
                               [1.47696749, 1.54609464, 1.70398688, 1.94690582]])

    Color_noise_white = np.array([[ 0.52651906,  0.55626041,  0.42810822,  0.17993373],
                                  [ 0.49084841,  0.46824137, -0.45950314, -0.30368259],
                                  [ 0.47391403,  0.46824137,  0.21482022, -0.30368259],
                                  [ 0.47391403,  0.52651906,  0.21482022,  0.42810822],
                                  [ 0.52795351,  0.55626041, -0.12620915,  0.17993373],
                                  [ 0.50716462,  0.49084841, -0.31563806, -0.45950314],
                                  [ 0.50716462,  0.52795351, -0.31563806, -0.12620915]])

    pm_noise = np.array([[0.78214647, 0.58558333, 0.08250894, 0.35462665],
                           [0.95896388, 0.78214647, 0.38327095, 0.08250894],
                           [0.88133414, 0.58558333, 1.12804452, 0.35462665],
                           [0.88133414, 0.95559041, 1.12804452, 1.23922934],
                           [0.98044892, 0.98755579, 1.17383784, 0.81952185],
                           [0.98044892, 0.95559041, 1.17383784, 1.23922934],
                           [0.98541874, 0.98755579, 0.74716073, 0.81952185],
                           [0.98541874, 0.95896388, 0.74716073, 0.38327095]])

    chaos_noise = np.array([[ 0.07471322,  0.02895051, -0.02780627, -1.7798434 ],
                           [ 0.07471322,  0.46756863, -0.02780627, -0.08975267],
                           [ 0.02908156,  0.02895051, -2.45732806, -1.7798434 ],
                           [ 0.02908156,  0.28014732, -2.45732806, -1.59004539],
                           [ 0.43151675,  0.46756863, -0.56256568, -0.08975267],
                           [ 0.43151675,  0.28014732, -0.56256568, -1.59004539]])
    
    return noise,Color_noise_pink,Color_noise_red,Color_noise_white,pm_noise,chaos_noise